# RDP Sensitive Settings Changed
Detects changes to RDP terminal service sensitive settings

## Rule Content
```
- title: RDP Sensitive Settings Changed
  id: 171b67e1-74b4-460e-8d55-b331f3e32d67
  description: Detects changes to RDP terminal service sensitive settings
  references:
  - https://blog.menasec.net/2019/02/threat-hunting-rdp-hijacking-via.html
  date: 2019/04/03
  author: Samir Bousseaden
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection_reg:
      EventID: 13
      TargetObject:
      - '*\services\TermService\Parameters\ServiceDll*'
      - '*\Control\Terminal Server\fSingleSessionPerUser*'
      - '*\Control\Terminal Server\fDenyTSConnections*'
    condition: selection_reg
  tags:
  - attack.defense_evasion
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*\\services\\TermService\\Parameters\\ServiceDll* OR *\\Control\\Terminal\ Server\\fSingleSessionPerUser* OR *\\Control\\Terminal\ Server\\fDenyTSConnections*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()